In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#Question1: How have movies performed over the years?
#time_series / group by genre 

/kaggle/input/movies/movies.csv


# * * * * * Fixing NAN values

In [2]:
#Checked for nan values in rows
file_path = "/kaggle/input/movies/movies.csv"
df = pd.read_csv(file_path)
df_row_mask= df.isna().any(axis=1)
rows_with_nan = df[df_row_mask]
rows_with_nan.head()


,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
16,Fame,R,Drama,1980,"May 16, 1980 (United States)",6.6,21000.0,Alan Parker,Christopher Gore,Eddie Barth,United States,NaN,21202829.0,Metro-Goldwyn-Mayer (MGM),134.0
19,Stir Crazy,R,Comedy,1980,"December 12, 1980 (United States)",6.8,26000.0,Sidney Poitier,Bruce Jay Friedman,Gene Wilder,United States,NaN,101300000.0,Columbia Pictures,111.0
24,Urban Cowboy,PG,Drama,1980,"June 6, 1980 (United States)",6.4,14000.0,James Bridges,Aaron Latham,John Travolta,United States,NaN,46918287.0,Paramount Pictures,132.0
25,Altered States,R,Horror,1980,"December 25, 1980 (United States)",6.9,33000.0,Ken Russell,Paddy Chayefsky,William Hurt,United States,NaN,19853892.0,Warner Bros.,102.0
26,Little Darlings,R,Comedy,1980,"March 21, 1980 (United States)",6.5,5100.0,Ron Maxwell,Kimi Peck,Tatum O'Neal,United States,NaN,34326249.0,Stephen Friedman/Kings Road Productions,96.0


In [3]:
# print column names with nan values
df_columns_mask= df.isna().any(axis=0)
df.columns[df_columns_mask]
    

Index(['rating', 'released', 'score', 'votes', 'writer', 'star', 'country',
       'budget', 'gross', 'company', 'runtime'],
      dtype='object')

In [4]:
#look at how many nan values in each column
columns = df.columns[df_columns_mask]
for col in columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")

Column rating has 77 NaN values
Column released has 2 NaN values
Column score has 3 NaN values
Column votes has 3 NaN values
Column writer has 3 NaN values
Column star has 1 NaN values
Column country has 3 NaN values
Column budget has 2171 NaN values
Column gross has 189 NaN values
Column company has 17 NaN values
Column runtime has 4 NaN values


In [5]:
#replace missing values in budget with mean for that year
mean_per_year = df.groupby('year')['budget'].transform('mean')
df['budget'] = df['budget'].fillna(mean_per_year)
for col in columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")

Column rating has 77 NaN values
Column released has 2 NaN values
Column score has 3 NaN values
Column votes has 3 NaN values
Column writer has 3 NaN values
Column star has 1 NaN values
Column country has 3 NaN values
Column budget has 0 NaN values
Column gross has 189 NaN values
Column company has 17 NaN values
Column runtime has 4 NaN values


In [6]:
#Drop too specific columns
#Drop nan values in the gross column because those will be irrelevant to analysis
new_df = df.dropna(subset=['gross'])
for col in columns:
    print(f"Column {col} has {new_df[col].isna().sum()} NaN values")

Column rating has 54 NaN values
Column released has 0 NaN values
Column score has 0 NaN values
Column votes has 0 NaN values
Column writer has 3 NaN values
Column star has 0 NaN values
Column country has 1 NaN values
Column budget has 0 NaN values
Column gross has 0 NaN values
Column company has 10 NaN values
Column runtime has 1 NaN values


In [7]:
# removes any other nan values as the number is relatively small
new_df = new_df.dropna()

In [8]:
for col in columns:
    print(f"Column {col} has {new_df[col].isna().sum()} NaN values")

Column rating has 0 NaN values
Column released has 0 NaN values
Column score has 0 NaN values
Column votes has 0 NaN values
Column writer has 0 NaN values
Column star has 0 NaN values
Column country has 0 NaN values
Column budget has 0 NaN values
Column gross has 0 NaN values
Column company has 0 NaN values
Column runtime has 0 NaN values


In [9]:
# replace dates with Months
months_order = ['Jan', 'Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

def replace_dates(text):
    for month in months_order:
        if month in text:
            return month
    return np.nan 
new_df['released'] = new_df['released'].apply(lambda x: replace_dates(x))
new_df['released'] = pd.Categorical(new_df['released'], categories=months_order, ordered=True)


In [10]:
NAreleased = new_df['released'].isna().sum()
NAreleased

7

In [11]:
new_df = new_df.dropna()

In [12]:
for col in columns:
    print(f"Column {col} has {new_df[col].isna().sum()} NaN values")

Column rating has 0 NaN values
Column released has 0 NaN values
Column score has 0 NaN values
Column votes has 0 NaN values
Column writer has 0 NaN values
Column star has 0 NaN values
Column country has 0 NaN values
Column budget has 0 NaN values
Column gross has 0 NaN values
Column company has 0 NaN values
Column runtime has 0 NaN values


In [13]:
new_df['date'] = new_df['released'].astype(str) + '-' + new_df['year'].astype(str)
new_df['date'] = pd.to_datetime(new_df['date'], format='%b-%Y')
new_order = ['date'] + [col for col in df.columns if col != 'date']
new_df = new_df[new_order]
new_df.head()

,date,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,1980-06-01,The Shining,R,Drama,1980,Jun,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,1980-07-01,The Blue Lagoon,R,Adventure,1980,Jul,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,1980-06-01,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,Jun,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,1980-07-01,Airplane!,PG,Comedy,1980,Jul,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,1980-07-01,Caddyshack,R,Comedy,1980,Jul,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


# * * * * *  Organising data into a time series

In [14]:
# Group by 'date', then calculate the mean of numeric columns
numeric_columns = new_df.select_dtypes(include='number')
grouped_df = new_df.groupby(['date'],observed = True)[numeric_columns.columns].mean()
grouped_df

,year,score,votes,budget,gross,runtime
date,,,,,,
1980-01-01,1980.0,5.150000,614.000000,1.210319e+07,2.107650e+06,104.000000
1980-02-01,1980.0,6.266667,18900.000000,8.851596e+06,1.575748e+07,100.166667
1980-03-01,1980.0,6.600000,4630.571429,1.323131e+07,1.769916e+07,101.285714
1980-04-01,1980.0,6.625000,8076.000000,1.832660e+07,2.672808e+06,138.500000
1980-05-01,1980.0,6.142857,23577.142857,9.408511e+06,1.572134e+07,96.857143
...,...,...,...,...,...,...
2020-04-01,2020.0,5.400000,2400.000000,5.753105e+07,3.661000e+03,98.000000
2020-06-01,2020.0,5.000000,294.000000,5.753105e+07,4.133780e+05,120.000000
2020-08-01,2020.0,6.800000,3700.000000,8.000000e+07,4.614216e+08,149.000000


In [15]:
def format_number(x):
    if isinstance(x, (int, float)):
        return f"{x:,.0f}"
    return x

In [16]:
numeric_columns = grouped_df.select_dtypes(include='number').columns.difference(['year'])
grouped_df[numeric_columns] = grouped_df[numeric_columns].map(format_number)
grouped_df



,year,score,votes,budget,gross,runtime
date,,,,,,
1980-01-01,1980.0,5,614,"12,103,191","2,107,650",104
1980-02-01,1980.0,6,"18,900","8,851,596","15,757,484",100
1980-03-01,1980.0,7,"4,631","13,231,307","17,699,157",101
1980-04-01,1980.0,7,"8,076","18,326,596","2,672,808",138
1980-05-01,1980.0,6,"23,577","9,408,511","15,721,338",97
...,...,...,...,...,...,...
2020-04-01,2020.0,5,"2,400","57,531,050","3,661",98
2020-06-01,2020.0,5,294,"57,531,050","413,378",120
2020-08-01,2020.0,7,"3,700","80,000,000","461,421,559",149


In [17]:
time_series = grouped_df.reset_index()
columns_to_convert = ['votes', 'gross', 'budget','runtime','score']


for col in columns_to_convert:
    time_series[col] = time_series[col].replace({'\$': '', ',': ''}, regex=True).astype(float)

In [18]:
#time_series.to_csv('movie_time_series.csv', index=False)

# * * * * * Organising data by year and country

In [19]:
cgrouped_df = new_df.groupby(['country','year']).agg({'gross': 'sum', 'budget': 'sum', 'score': 'mean', 'runtime':'mean'}).reset_index()
numeric_columns = cgrouped_df.select_dtypes(include='number').columns.difference(['year'])
cgrouped_df[numeric_columns] = cgrouped_df[numeric_columns].map(format_number)
cgrouped_df
#cgrouped_df.to_csv('movie_grouped_by_country.csv', index=False)

,country,year,gross,budget,score,runtime
0,Argentina,1985,"69,832","12,454,184",8,112
1,Argentina,2000,"12,413,888","1,500,000",8,114
2,Argentina,2001,"270,811","38,833,133",7,103
3,Argentina,2004,"57,663,711","44,286,880",8,126
4,Argentina,2009,"35,079,650","2,000,000",8,129
...,...,...,...,...,...,...
624,West Germany,1984,"22,458,959","39,267,319",8,124
625,West Germany,1986,"7,153,487","12,522,647",8,130
626,West Germany,1987,"7,135,886","25,602,642",8,112
627,West Germany,1990,"2,293,625","18,340,735",7,92


* * * * * Organising data by genre

In [20]:
ggrouped_df = new_df.groupby(['genre','year']).agg({'gross': 'sum', 'budget': 'sum', 'score': 'mean', 'runtime':'mean'}).reset_index()

numeric_columns = ggrouped_df.select_dtypes(include='number').columns.difference(['year'])
ggrouped_df[numeric_columns] = cgrouped_df[numeric_columns].map(format_number)
ggrouped_df = ggrouped_df.reset_index()
ggrouped_df
#ggrouped_df.to_csv('movie_grouped_by_genre.csv', index=False)

,index,genre,year,gross,budget,score,runtime
0,0,Action,1980,"69,832","12,454,184",8,112
1,1,Action,1981,"12,413,888","1,500,000",8,114
2,2,Action,1982,"270,811","38,833,133",7,103
3,3,Action,1983,"57,663,711","44,286,880",8,126
4,4,Action,1984,"35,079,650","2,000,000",8,129
...,...,...,...,...,...,...,...
401,401,Thriller,2015,"1,055,995","18,340,735",8,158
402,402,Thriller,2019,"242,623","3,000,000",8,104
403,403,Western,1981,"40,185,556","7,000,000",8,121
404,404,Western,1982,"4,657,705","30,197,324",8,100


In [21]:
filtered_df = new_df[new_df['genre'] == 'Musical']
filtered_df


,date,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
7613,2019-05-01,42nd Street: The Musical,Not Rated,Musical,2019,May,7.9,321.0,Mark Bramble,Mark Bramble,Bonnie Langford,United Kingdom,5.740129e+07,2217255.0,Julian Marsh Pictures,155.0
